In [1]:
import pandas as pd

from country_abbrev import *
from country_language import *
from pytrends.request import TrendReq

import os
import pycountry

In [2]:
os.getcwd()

"g:\\Mi unidad\\BSE - DSDM\\Master's project\\refugees\\notebooks\\trends"

In [3]:
currencies = pd.read_excel("countries-codes-and-currencies-(2020).xlsx")

In [4]:
currencies.head()

,Country Code,Country Name,WHO Region,World Bank Income Group (2018),Currency Name,Currency Code
0,AFG,Afghanistan,EMR,Low,Afghanistan Afghani,AFN
1,ALB,Albania,EUR,Upper‐middle,Albanian Lek,ALL
2,DZA,Algeria,AFR,Upper‐middle,Algerian Dinar,DZD
3,AND,Andorra,EUR,High,Euro,EUR
4,AGO,Angola,AFR,Lower‐middle,Angolan Kwanza,AON


In [5]:
currencies.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Country Code                    194 non-null    object
 1   Country Name                    194 non-null    object
 2   WHO Region                      194 non-null    object
 3   World Bank Income Group (2018)  194 non-null    object
 4   Currency Name                   194 non-null    object
 5   Currency Code                   194 non-null    object
dtypes: object(6)
memory usage: 9.2+ KB


In [6]:
keywords_list= list(set(currencies["Currency Name"]))
len(keywords_list)

# First search as it comes, then we can translate for each origin lang

141

In [10]:
keywords_list = ['Bolivar Fuerte',
 'Lari',
 'Cuban Peso',
 'Nuevo Sol',
 'Kuwaiti Dinar',
 'Naira',
 'Moroccan Dirham',
 'CFA Franc',
 'Kyat',
 'Nepalese Rupee',
 'Bhat',
 'Australian Dollar',
 'Swiss Franc',
 'Algerian Dinar',
 'Barbados Dollar',
 'Tanzanian Shilling',
 'Manat',
 'Guyana Dollar',
 'Romania Leu',
 'Dong',
 'Haitian Gourde',
 'Kwacha',
 'Jordanian Dinar',
 'Indian Rupee',
 'Sum',
 'Nicaraguan Cordoba Oro',
 'Icelandic Krona',
 'Fiji Dollar',
 'Albanian Lek',
 'US Dollar',
 'Congolese Franc',
 'Cape Verde Escudo',
 'Zloty',
 'Belize Dollar',
 'New Zealand Dollar',
 'Ringgit',
 'Rouble',
 'Omani Rial',
 'Ngultrum',
 'Taka',
 'Euro',
 'Norway Krone',
 'Ghana Cedi',
 'Kina',
 'Rufiyaa',
 'Pound Sterling',
 'Gambian Dalasi',
 'Belarusian Ruble',
 'Riel',
 'Leone',
 'Philippines Peso',
 'Brunei Dollar',
 'Namibian Dollar',
 "Pa'anga",
 'Botswana Pula',
 'Moldova Leu',
 'Bulgarian Lev',
 'Baharaini Dinar',
 'Vatu',
 'Malawian Kwacha',
 'Guatemalan Quetzal',
 'Lebanese Pound',
 'Mauritius Rupee',
 'Nakfa',
 'Chilean Peso',
 'North Korean Won',
 'Turkish Lira',
 'Trinidad and Tobago Dollar',
 'New Sheqel',
 'Ethiopian Birr',
 'Egyptian Pound',
 'Syrian Pound',
 'Hungarian Forint',
 'Qatar Rial',
 'Tenge',
 'Rwandan Franc',
 'Indonesian Rupiah',
 'New Ouguiya',
 'Danish Krone',
 'Denar',
 'Balboa',
 'Yemeni Rial',
 'Won',
 'New Dobra',
 'Afghanistan Afghani',
 'Guinea Franc',
 'Burundi Franc',
 'Colombian Peso',
 'Malagasy Ariary',
 'Japanese Yen',
 'Jamaican Dollar',
 'Eastern Caribbean Dollar',
 'Lilangeni',
 'Serbian Dinar',
 'Singapore Dollar',
 'Kenyan Shilling',
 'Iraqi Dinar',
 'Pakistani Rupee',
 'Djibouti Franc',
 'Tala',
 'Lesotho Loti',
 'Bahamian Dollar',
 'Sri Lanka Rupee',
 'Somalian Shilling',
 'Ugandan shilling',
 'Saudi Riyal',
 'Solomon Dollar',
 'Somoni',
 'Hryvnia',
 'Costa Rican Colone',
 'Tugrik',
 'Yuan Renminbi',
 'Iranian Rial',
 'Tunisian Dinar',
 'Liberian Dollar',
 'Swedish Krona',
 'Dram',
 'Seychelles Rupee',
 'Mexican Peso',
 'Angolan Kwanza',
 'Surinamese Dollar',
 'Canadian Dollar',
 'Uruguayan Peso',
 'South Sudanese Pound',
 'Honduran Lempira',
 'Convertible Mark',
 'Guarani',
 'Kip',
 'Comoros Franc',
 'Dominican Peso',
 'Som',
 'Kuna',
 'U.A.Emirates Dirham']

In [11]:
countries_iso3 = currencies["Country Code"]
countries = [pycountry.countries.get(alpha_3=code).alpha_2 for code in countries_iso3]

In [13]:
from pytrends.request import TrendReq

import time
# Set up the Google Trends API object
pytrends = TrendReq()

# Initialize an empty DataFrame to store the results
results = pd.DataFrame()

# Loop through regions and keywords
for keyword in keywords_list:
    for country in countries:
        print("Searching: " + str(keyword) + " in " + str(country))
        time.sleep(1)
        # Build the payload with the selected region, keyword and date range
        pytrends.build_payload(kw_list=[keyword], timeframe='all', geo=str(country))
        # Get the interest over time data for the selected keyword
        interest_over_time_df = pytrends.interest_over_time()

        # Check if the DataFrame is not empty
        if not interest_over_time_df.empty:
            interest_over_time_df= interest_over_time_df.reset_index()
            interest_over_time_df['trends_index'] = interest_over_time_df.iloc[:, 1] 
            interest_over_time_df = interest_over_time_df.drop(interest_over_time_df.columns[1], axis=1)

            interest_over_time_df = interest_over_time_df.drop(columns=["isPartial"], axis=1)
            interest_over_time_df["keyword"]=keyword
            interest_over_time_df["region"] = country
            
            # Append the results to the main dataframe
            results = pd.concat([results, interest_over_time_df])

Searching: Bolivar Fuerte in AF
Searching: Bolivar Fuerte in AL
Searching: Bolivar Fuerte in DZ
Searching: Bolivar Fuerte in AD
Searching: Bolivar Fuerte in AO
Searching: Bolivar Fuerte in AG
Searching: Bolivar Fuerte in AR
Searching: Bolivar Fuerte in AM
Searching: Bolivar Fuerte in AU
Searching: Bolivar Fuerte in AT
Searching: Bolivar Fuerte in AZ
Searching: Bolivar Fuerte in BS
Searching: Bolivar Fuerte in BH
Searching: Bolivar Fuerte in BD
Searching: Bolivar Fuerte in BB
Searching: Bolivar Fuerte in BY
Searching: Bolivar Fuerte in BE
Searching: Bolivar Fuerte in BZ
Searching: Bolivar Fuerte in BJ
Searching: Bolivar Fuerte in BT
Searching: Bolivar Fuerte in BO
Searching: Bolivar Fuerte in BA
Searching: Bolivar Fuerte in BW
Searching: Bolivar Fuerte in BR
Searching: Bolivar Fuerte in BN
Searching: Bolivar Fuerte in BG
Searching: Bolivar Fuerte in BF
Searching: Bolivar Fuerte in BI
Searching: Bolivar Fuerte in CV
Searching: Bolivar Fuerte in KH
Searching: Bolivar Fuerte in CM
Searchin

ReadTimeout: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)

In [14]:
results

,date,trends_index,keyword,region
0,2004-01-01,0,Bolivar Fuerte,AR
1,2004-02-01,0,Bolivar Fuerte,AR
2,2004-03-01,0,Bolivar Fuerte,AR
3,2004-04-01,0,Bolivar Fuerte,AR
4,2004-05-01,0,Bolivar Fuerte,AR
...,...,...,...,...
228,2023-01-01,0,Cuban Peso,IN
229,2023-02-01,0,Cuban Peso,IN
230,2023-03-01,0,Cuban Peso,IN
231,2023-04-01,0,Cuban Peso,IN


In [15]:
results.to_csv("currency_partial results/currency3.csv")